# Dynamic Gating Quickstart
This notebook shows how to read an nwb for the dynamic gating experiment. The focus is on aligning neural data to visual and optotagging stimuli. This project shares many similarities with the Visual Behavior Neuropixels, so users are encouraged to check out the documentation for those for additional information and context

In [6]:
import pynwb
from allensdk.brain_observatory.ecephys.dynamic_gating_ecephys_session import DynamicGatingEcephysSession
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pathlib

# Metadata Tables

Metadata tables have been provided for easy access to do analysis across sessions. Theses tables can be found under the metdata_tables folder in the repository. There are four tables

* Sessions table: Contains relevant information on each session, such as areas hit, probes inserted and tracked, unit counts, etc.
* Probes table: Contains metadata on all probes for all sessions in the dataset
* Channels table: Contain metadata on electrodes on the probes. This includes coordinates for each channel in the Allen Common Coordinate Framework (CCF) and the CCF brain area
* Units table: Contains all unit metrics from sorting for all sessions including firing rate, peak channel, etc.

In [17]:
sessions_metadata_table = pd.read_csv(pathlib.Path('../metadata_tables/dynamic_gating_session_metadata.csv'))
probes_table = pd.read_csv(pathlib.Path('../metadata_tables/probes_table.csv'))
channels_table = pd.read_csv(pathlib.Path('../metadata_tables/channels_table.csv'))
units_table = pd.read_csv(pathlib.Path('../metadata_tables/units_table.csv'))

In [ ]:
# filter out channels whose probes have not been traced to the ccf, channels out of the brain, and channels in white matter
channels_table = channels_table[(channels_table['structure_acronym'] != 'out of brain') & (channels_table['structure_acronym'] != 'No Area') & 
                    (channels_table['structure_acronym'] != 'Track not annotated')]

# Read From NWB

This section shows how to read a nwb file for a single session experiment. Find the session you want to look at, and just give the path to it as seen below. Then, we can use the SDK in a very similar manner to the Visual Behavior Neuropixels to access relevant information.

In [ ]:
session = 'sub-608671_ses-20220518T214848.nwb'
# REPLACE WITH CORRESPONDING PATH
nwb_file_asset = pynwb.NWBHDF5IO(f'../data/sub-608671/{session}', mode='r', load_namespaces=True)
nwb_file = nwb_file_asset.read()
dynamic_gating_session = DynamicGatingEcephysSession.from_nwb(nwb_file)

We can get a high-level summary of the session by accessing the metadata attribute

In [ ]:
dynamic_gating_session.metadata
session_id = dynamic_gating_session.metadata['ecephys_session_id']

We will now get the units and channels for this session. Then we will merge them to get the CCF areas and coordinates for each unit

In [ ]:
units = dynamic_gating_session.get_units()
units.columns

In [ ]:
units = dynamic_gating_session.get_units()
channels = dynamic_gating_session.get_channels()
# filter out channels whose probes have not been traced to the ccf, channels out of the brain, and channels in white matter
channels = channels[(channels['structure_acronym'] != 'out of brain') & (channels['structure_acronym'] != 'No Area') & 
                    (channels['structure_acronym'] != 'Track not annotated')]

units_channels = units.merge(channels, left_on='peak_channel_id', right_index=True)
units_channels

Now we will look at the brain structures that were recorded during this session

In [ ]:
units_channels.value_counts('structure_acronym')

# PSTH for image changes
Next, we will grab spike times and calculate the change response for 'good' units. The filtering of the units will depend on the analysis done

In [ ]:
spike_times = dynamic_gating_session.spike_times

#first let's sort our units by depth
units_channels = units_channels.sort_values('probe_vertical_position', ascending=False)

#now we'll filter them
good_unit_filter = ((units_channels['snr']>1)&
                    (units_channels['isi_violations']<1)&
                    (units_channels['firing_rate']>0.1))

good_units = units_channels.loc[good_unit_filter]

We can get the times when the image changes occurred from the stimulus presentations table. For now, we'll only take the image changes shown during the active behavior block

In [ ]:
stimulus_presentations = dynamic_gating_session.stimulus_presentations
change_times = stimulus_presentations[stimulus_presentations['active']&
                            stimulus_presentations['is_change']]['start_time'].values

In [ ]:
#Convenience function to compute the PSTH
def makePSTH(spikes, startTimes, windowDur, binSize=0.001):
    bins = np.arange(0,windowDur+binSize,binSize)
    counts = np.zeros(bins.size-1)
    for i,start in enumerate(startTimes):
        startInd = np.searchsorted(spikes, start)
        endInd = np.searchsorted(spikes, start+windowDur)
        counts = counts + np.histogram(spikes[startInd:endInd]-start, bins)[0]
    
    counts = counts/startTimes.size
    return counts/binSize, bins

We'll include enough time in our plot to see three image responses: the pre-change image response, the change response and the post-change response

In [ ]:
# function to look at response for any given area
def get_area_response(area_of_interest:str, good_units:pd.DataFrame, spike_times:dict, time_before_change:float=1) -> tuple[np.ndarray, ...]:
    #Here's where we loop through the units in our area of interest and compute their PSTHs
    area_change_responses = []
    area_units = good_units[good_units['structure_acronym'].str.contains(area_of_interest)]
    duration = 2.5
    for iu, unit in area_units.iterrows():
        unit_spike_times = spike_times[iu]
        unit_change_response, bins = makePSTH(unit_spike_times, 
                                              change_times-time_before_change, 
                                              duration, binSize=0.01)
        area_change_responses.append(unit_change_response)
    area_change_responses = np.array(area_change_responses)
    
    return area_change_responses, bins
    

In [ ]:
area_of_interest = 'VISp'
time_before_change = 1
area_change_responses, bins = get_area_response(area_of_interest, good_units, spike_times, time_before_change=time_before_change)

#Plot the results
fig, ax = plt.subplots(1,2)
fig.set_size_inches([12,4])

clims = [np.percentile(area_change_responses, p) for p in (0.1,99.9)]
im = ax[0].imshow(area_change_responses, clim=clims)
ax[0].set_title('Active Change Responses for {}'.format(area_of_interest))
ax[0].set_ylabel('Unit number, sorted by depth')
ax[0].set_xlabel('Time from change (s)')
ax[0].set_xticks(np.arange(0, bins.size-1, 20))
_ = ax[0].set_xticklabels(np.round(bins[:-1:20]-time_before_change, 2))

ax[1].plot(bins[:-1]-time_before_change, np.mean(area_change_responses, axis=0), 'k')
ax[1].set_title('{} population active change response (n={})'\
                .format(area_of_interest, area_change_responses.shape[0]))
ax[1].set_xlabel('Time from change (s)')
ax[1].set_ylabel('Firing Rate')

# save
fig.savefig('../results/{}_test_response.png'.format(session_id))

# Plot Receptive Fields

Now, we will look at plotting the receptive fields. First we need to get stimulus presentation data for the receptive field mapping stimulus (gabors).

In [ ]:
receptive_field_stim_table = stimulus_presentations[stimulus_presentations['stimulus_name'].str.contains('gabor')]
xs = np.sort(receptive_field_stim_table.position_x.unique()) #positions of gabor along azimuth
ys = np.sort(receptive_field_stim_table.position_y.unique()) #positions of gabor along elevation

In [ ]:
# helper function to find the receptive fields
def find_receptive_field(spikes, xs, ys) -> np.ndarray:
    unit_receptive_field = np.zeros([ys.size, xs.size])
    for ix, x in enumerate(xs):
        for iy, y in enumerate(ys):
            stim_times = receptive_field_stim_table[(receptive_field_stim_table.position_x==x)
                                      &(receptive_field_stim_table.position_y==y)]['start_time'].values
            unit_response, bins = makePSTH(spikes, 
                                          stim_times+0.01, 
                                          0.2, binSize=0.001)
            unit_receptive_field[iy, ix] = unit_response.mean()
    return unit_receptive_field

In [ ]:
# function to get receptive fields given any area
def get_receptive_fields(area_of_interest:str) -> np.ndarray:
    area_receptive_fields = []
    area_units = good_units[good_units['structure_acronym'].str.contains(area_of_interest)]
    
    for iu, unit in area_units.iterrows():
        unit_spike_times = spike_times[iu]
        unit_receptive_field = find_receptive_field(unit_spike_times, xs, ys)
        area_receptive_fields.append(unit_receptive_field)
    
    return area_receptive_fields

In [ ]:
area_of_interest = 'VISp'
area_receptive_fields = get_receptive_fields(area_of_interest)

fig, axes = plt.subplots(int(len(area_receptive_fields)/10)+1, 10)
fig.set_size_inches(12, 8)
for irf, rf in enumerate(area_receptive_fields):
    ax_row = int(irf/10)
    ax_col = irf%10
    axes[ax_row][ax_col].imshow(rf, origin='lower')
for ax in axes.flat:
    ax.axis('off')

# Optotagging

Let's look at the optotagging table now and plot PSTHs triggered on the laser onset.

In [ ]:
opto_table = dynamic_gating_session.optotagging_table
time_before = 0.01 # seconds to take before the laser start for PSTH
    
opto_table.head()

In [ ]:
# function to get opto response for any given probe
def get_opto_response(probe_id:int) -> tuple[np.ndarray, ...]:
    duration = opto_table.duration.min() #get the short pulses
    level = opto_table.level.max() #and the high power trials

    cortical_units = good_units[good_units['probe_id'] == probe_id]

    opto_times = opto_table.loc[(opto_table['duration']==duration)&
                                (opto_table['level']==level)]['start_time'].values
    
    duration = 0.03 # total duration of trial for PSTH in seconds
    binSize = 0.001 # 1ms bin size for PSTH
    
    opto_response = []
    unit_id = []
    for iu, unit in cortical_units.iterrows():
        unit_spike_times = spike_times[iu]
        unit_response, bins = makePSTH(unit_spike_times, 
                              opto_times-time_before, duration, 
                              binSize=binSize)

        opto_response.append(unit_response)
        unit_id.append(iu)

    opto_response = np.array(opto_response)
    
    return opto_response, bins

We'll use the probes data from the session and look at the optotagging on a per probe basis

In [ ]:
dynamic_gating_session.probes

In [ ]:
probes = dynamic_gating_session.probes
probe_index = 3

probe_name = probes.iloc[probe_index]['name']
probe_id = probes.index[probe_index]

opto_response, bins = get_opto_response(probe_id)

fig, ax = plt.subplots()
fig.set_size_inches((5,10))
fig.suptitle('Optotagging: ' + str(dynamic_gating_session.metadata['ecephys_session_id'])
             + ' ' + dynamic_gating_session.metadata['full_genotype'] + ' ' + probe_name)
im = ax.imshow(opto_response, 
               origin='lower', aspect='auto',
               )
min_clim_val = 0
max_clim_val = 250
im.set_clim([min_clim_val, max_clim_val])    
[ax.axvline(bound, linestyle=':', color='white', linewidth=1.0)\
     for bound in [10, 19]]
ax.set_xlabel('Time from laser onset (ms)')
ax.set_ylabel('Unit number')
ax.set_xticks(1000*bins[:-1:5])

time_labels = np.round(1000*(bins[:-1:5]-time_before), 0)
_=ax.set_xticklabels(time_labels)

Let's plot the response to the laser over the population

In [ ]:
baseline_window = slice(0, 9)  # baseline epoch
response_window = slice(11,18) # laser epoch

response_magnitudes = np.mean(opto_response[:, response_window], axis=1) \
                    - np.mean(opto_response[:, baseline_window], axis=1)

In [ ]:
fig, axes = plt.subplots(1,2)
fig.set_size_inches(10, 5)

# Plot scatter of opto rate vs baseline rate
axes[0].plot(np.mean(opto_response[:, baseline_window], axis=1),
         np.mean(opto_response[:, response_window], axis=1), 'k.', alpha=0.2)
axes[0].set_xlim([-10, 200])
axes[0].set_ylim([-10, 400])
axes[0].set_aspect('equal')
axes[0].set_ylabel('response rate (Hz)')
axes[0].set_xlabel('baseline rate (Hz)')

# Plot histogram of opto-evoked rate (note log yscale)
_ = axes[1].hist(response_magnitudes, bins=20)
axes[1].set_yscale('log')
axes[1].set_xlabel('Opto-evoked rate (Hz)')
axes[1].set_ylabel('Unit Count')